In [5]:
import requests
from datetime import datetime
import time
import pandas as pd

# 2.3 Request Module
- https://docs.github.com/en/rest/repos/repos?apiVersion=2022-11-28

In [1]:
## requests.get(), response.status_code
import requests
response= requests.get('https://api.github.com/repositories',
                      headers={'Accept':'application/vnd.github+json'})
print(response.status_code)

200


In [12]:
## response.headers
print(response.encoding)
print(response.headers['Content-Type'])
print(response.headers['server'])
print(response.headers['Access-Control-Expose-Headers'])
print(response.headers)

utf-8
application/json; charset=utf-8
GitHub.com
ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset
{'Server': 'GitHub.com', 'Date': 'Tue, 30 Jan 2024 10:23:37 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': 'W/"3c54a93639294e719c717ee9fb29aa31617be2a77aed1b49a769e1b1c9dd5ef5"', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Link': '<https://api.github.com/repositories?since=369>; rel="next", <https://api.github.com/repositories{?since}>; rel="first"', 'x-github-api-version-selected': '2022-11-28', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-Rat

In [21]:
## response를 json 객체로 변환하기
import json
print(len(response.json())) # 100: 100개의 데이터가 있음, response.json()
print(json.dumps(response.json()[0], indent=2)[:200])

100
{
  "id": 1,
  "node_id": "MDEwOlJlcG9zaXRvcnkx",
  "name": "grit",
  "full_name": "mojombo/grit",
  "private": false,
  "owner": {
    "login": "mojombo",
    "id": 1,
    "node_id": "MDQ6VXNlcjE=",



In [22]:
## query 매개변수를 이용하여 필요한 정보 search
response=requests.get('https://api.github.com/repositories',
                      headers={'Accept':'application/vnd.github+json'},
                     params= {'q':'data+language:python'})
print(response.status_code)

200


## 레포지토리의 댓글 모니터링

In [32]:
response= requests.get('https://api.github.com/repos/pytorch/pytorch/issues/comments')
print(response.status_code)
print(len(response.json()))
print(response.json()[29])
## 왜 30개의 댓글만 가져온 것일까?

200
30
{'url': 'https://api.github.com/repos/pytorch/pytorch/issues/comments/246085619', 'html_url': 'https://github.com/pytorch/pytorch/issues/20#issuecomment-246085619', 'issue_url': 'https://api.github.com/repos/pytorch/pytorch/issues/20', 'id': 246085619, 'node_id': 'MDEyOklzc3VlQ29tbWVudDI0NjA4NTYxOQ==', 'user': {'login': 'apaszke', 'id': 4583066, 'node_id': 'MDQ6VXNlcjQ1ODMwNjY=', 'avatar_url': 'https://avatars.githubusercontent.com/u/4583066?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/apaszke', 'html_url': 'https://github.com/apaszke', 'followers_url': 'https://api.github.com/users/apaszke/followers', 'following_url': 'https://api.github.com/users/apaszke/following{/other_user}', 'gists_url': 'https://api.github.com/users/apaszke/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/apaszke/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/apaszke/subscriptions', 'organizations_url': 'https://api.github.com/users/apaszke/orgs'

In [28]:
## 페이지 매기기
response.links
# 다음 페이지가 존재했음. 1페이지당 30개씩만 출력

{'next': {'url': 'https://api.github.com/repositories/65600975/issues/comments?page=2',
  'rel': 'next'},
 'last': {'url': 'https://api.github.com/repositories/65600975/issues/comments?page=1000',
  'rel': 'last'}}

### function: get_all_pages(url, params, headers)

In [34]:
def get_all_pages(url, params=None, headers=None):
    output_json=[]
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json() # reponse -> list( dict{}, dict{},...)
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json+=get_all_pages(next_url, params, headers) # 재귀 함수를 이용하여 다음 페이지가 있을 때까지 무한 반복시킴
    return output_json

In [57]:
url="https://api.github.com/repos/pytorch/pytorch/issues/comments"
params={"since":"2022-01-31T00:00:00Z",
       "sorted":"created", "direction":"desc"}
headers={'Accept':'application/vnd.github+json'}

start=datetime.now()
output=get_all_pages(url,params,headers)
lapse=  datetime.now()-start
print("경과시간: ", lapse)
# print("output",output[:2])
df=pd.DataFrame(output)
display(df.sample(1))
print("추출된 댓글의 수",df['body'].count())
display(df[['id','created_at','body']].sample(1))

경과시간:  0:00:02.041605


,url,html_url,issue_url,id,node_id,user,created_at,updated_at,author_association,body,reactions,performed_via_github_app
12,https://api.github.com/repos/pytorch/pytorch/i...,https://github.com/pytorch/pytorch/pull/115539...,https://api.github.com/repos/pytorch/pytorch/i...,1916501942,IC_kwDOA-j9z85yO3-2,"{'login': 'pytorchmergebot', 'id': 97764156, '...",2024-01-30T10:10:58Z,2024-01-30T10:10:58Z,COLLABORATOR,@pytorchbot started a rebase job onto [refs/re...,{'url': 'https://api.github.com/repos/pytorch/...,None


추출된 댓글의 수 60


,id,created_at,body
25,1916384722,2024-01-30T09:12:04Z,@pytorchbot merge


In [52]:

before=datetime.now()
time.sleep(5)
lapse=datetime.now()-before

In [54]:
print(lapse)

0:00:05.000577


In [60]:
## 속도제한 해결: 
# X-Ratelimit-Limit
# X-Ratelimit-Remaining
# X-Ratelimit-Reset

url="https://api.github.com/repos/pytorch/pytorch/issues/comments"
response=requests.get(url)

print('X-Ratelimit-Limit: ', response.headers['X-Ratelimit-Limit'])
print('X-Ratelimit-Remaining: ', response.headers['X-Ratelimit-Remaining'])

print('Rate Limits reset at: ', datetime.fromtimestamp(int(response.headers['X-Ratelimit-Reset'])).strftime('%c'))

X-Ratelimit-Limit:  60
X-Ratelimit-Remaining:  58
Rate Limits reset at:  Tue Jan 30 22:42:58 2024


### function handle_rate_limits(response) return boolean

In [2]:
def handle_rate_limits(response):
    now=datetime.now()
    reset_time = datetime.fromtimestamp(int(response.headers['X-RateLimit-Reset'])) # api 호출 가능 개수가 리셋되는 시간
    remaining_requests= response.headers['X-Ratelimit-Remaining'] # 남은 호출 가능 횟수
    remaining_time= (reset_time-now).total_seconds() # 리셋까지 남은 시간
    intervals= remaining_time/(1.0 + int(remaining_requests)) # 남은 시간 / 남은 호출+1 => 매 호출이 일어날 시간
    print('Sleeping for', intervals)
    time.sleep(intervals)
    return True

In [14]:
## Retry Strategy
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

retry_strategy = Retry(total=5, status_forcelist=[500,503,504], backoff_factor=1)
# status_forcelist: 어떠한 상태일 경우에 retry를 시도하는지를 지정
# backoff_factor: 서버에 대한 호출 시간을 기하급수적으로 늘려서 지연시키기

retry_adapter=HTTPAdapter(max_retries=retry_strategy)

http=requests.Session()
http.mount("https://", retry_adapter)
http.mount("http://", retry_adapter)

url='https://api.github.com/search/repositories'
params={'q':'data+language:python'}

response= http.get(url, params=params)

for item in response.json()['items'][:5]:
    print(item['name'])

awesome
kaitai_struct
data-structures-and-algorithms
Python-Roadmap
Sports_Data_Reference


In [16]:
print(len(response.json()['items']))
print(type(response.json()['items']))

30
<class 'list'>


## Total Process

In [19]:
# 1. Prepare retry strategy
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

retry_strategy = Retry(
    total=5,
    status_forcelist=[500, 503, 504],
    backoff_factor=1
)

retry_adapter = HTTPAdapter(max_retries=retry_strategy)

http = requests.Session()
http.mount("https://", retry_adapter)
http.mount("http://", retry_adapter)

# 2. Get all pages
def get_all_pages(url, param=None, header=None):
    output_json = []
    response = http.get(url, params=param, headers=header)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if (next_url is not None) and (handle_rate_limits(response)):  # 3. Handle limits
                output_json += get_all_pages(next_url, param, header)
    return output_json


In [ ]:
out = get_all_pages("https://api.github.com/repos/pytorch/pytorch/issues/comments", param={'since': '2023-12-01T00:00:01Z'})
df = pd.DataFrame(out)
print(df)

# 2.4 Twitter and python-twitter

- API KEY: dYVvLwbQUZnpfsjovN12mji3v
- API KEY SECRET: jW7M7XIrPgJkQQOk7ZuEkFOOR2VceSsBMWJzBEoLRQ2GCppUQI

- Searching posts in twitter
- search method가 사라져서 그냥 tweepy api 보고 따라해보기
- ap 통신

- tweepy 대신 python-twitter library 사용
- tweeter api 접근 권한 문제 때문에 포기. 한전 api랑 연합뉴스 api 이용해서 해보자

!pip install tweepy

In [14]:
import tweepy
app_api_key='dYVvLwbQUZnpfsjovN12mji3v'
app_api_secret_key='jW7M7XIrPgJkQQOk7ZuEkFOOR2VceSsBMWJzBEoLRQ2GCppUQI'


In [13]:
api.user_timeline(screen_name='The Associated Press', count=2)

Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product

!pip install python-twitter

In [22]:
import tweepy

# 트위터 API 키 설정
twitter_consumer_key = "yr2YMKX5GWqDxMVokRDZZKZ89"
twitter_consumer_secret = "1INBPDgLp6k97ciQCPlYzUddy5NDRbDcbHgHs8AywiPeovS2oA"
twitter_access_token = "1752588912676196352-VdW4akU7OeFejMgDHNaaWZdcfhedMo"
twitter_access_secret = "co5ZjYrJJG2RX86iNtEt1jLU1R1xUl1KfSBY6tImZsihM"

# API 인증
auth = tweepy.OAuthHandler(
    consumer_key=twitter_consumer_key,
    consumer_secret=twitter_consumer_secret
)
auth.set_access_token(twitter_access_token, twitter_access_secret)
api = tweepy.API(auth)

# 특정 계정의 타임라인 가져오기 -> 액세스 권한 없어서 불가
# account = "@TheBlueHouseKR"
# statuses = api.user_timeline(screen_name=account, count=200, include_rts=True, exclude_replies=False)

# # 트윗 텍스트 출력
# for status in statuses:
#     print(status.text)

# 특정 검색어로 트윗들을 가져오기 -> 액세스 권한 없음
# query = "solar energy"
# search_results = api.search_tweets(q=query, count=100)
# for tweet in search_results:
#     print(tweet.text)

# 특정 사용자의 실시간 트윗 가져오기 
user_screen_name = "AP"  # 사용자의 트위터 아이디
user_tweets = api.user_timeline(screen_name=user_screen_name)

# 트윗 텍스트 출력
for tweet in user_tweets:
    print(tweet.text)

Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product

# 2.4 한전 api 이용하여 정보 추출하기

In [1]:
import requests
apiKey='JydSLKAY7G2jdJ699nT03F5Lw922S1zpCWl91c4y'
settings={'year':'2023', 'month':'07', 'metroCd':'11', 'returnType':'json'}

url=f"https://bigdata.kepco.co.kr/openapi/v1/powerUsage/houseAve.do?year={settings['year']}&month={settings['month']}&metroCd={settings['metroCd']}&apiKey={apiKey}&returnType={settings['returnType']}"
response=requests.get(url)
print(response.status_code)

200


In [2]:
import json
response_list=response.json()['data']
for i in range(len(response_list)):
    if response_list[i]['city']=='광진구':
        print(i,response_list[i])

6 {'year': '2023', 'month': '07', 'metro': '서울특별시', 'city': '광진구', 'houseCnt': 187683, 'powerUsage': 245.38, 'bill': 36337}


# 2.4 뉴욕타임즈 api 이용하여 정보 추출하기

In [13]:
import json
import os
os.getcwd()
with open('../nyt_key.txt', 'r', encoding='utf-8') as f:
    key=f.read()

print(json.loads(key))
print(type(json.loads(key)))

{'api_key': 'hbhZf4CJove7aZhcx7eGFvdNxwnzasOg'}
<class 'dict'>


In [16]:
import requests
# 뉴욕 타임즈 API 키를 입력하세요
api_key = json.loads(key)['api_key']

url=f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q=north+korea&api-key={api_key}"
# 검색 조건을 설정합니다 (헤드라인에 "북한"이 포함된 기사)
search_params = {
    'q': '북한',
    'begin_date': '20230101',
    'end_date': '20231231',
}

# 기사를 검색합니다
response=requests.get(url)

# # 검색 결과를 출력합니다
# for article in articles['response']['docs']:
#     print(f"제목: {article['headline']['main']}")
#     print(f"URL: {article['web_url']}")
#     print("-" * 50)

In [18]:
response.headers

{'Date': 'Wed, 31 Jan 2024 11:53:05 GMT', 'Content-Type': 'application/json', 'Content-Length': '227367', 'Connection': 'keep-alive', 'Server': 'nginx', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Access-Control-Allow-Headers': 'Accept, Content-Type, Origin, X-Forwarded-For, X-Prototype-Version, X-Requested-With', 'Access-Control-Allow-Methods': 'GET, OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Length, X-JSON'}

In [19]:
response.json()

{'status': 'OK',
 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.',
 'response': {'docs': [{'abstract': 'Plus, when A.I. generates copyrighted images.',
    'web_url': 'https://www.nytimes.com/2024/01/25/briefing/north-korea-gaza-cease-fire-south-korea-lawmaker-attacked.html',
    'snippet': 'Plus, when A.I. generates copyrighted images.',
    'lead_paragraph': 'North Korea’s leader, Kim Jong-un, could take some form of lethal military action against South Korea in the coming months after having shifted his policy to one of open hostility, U.S. officials say.',
    'source': 'The New York Times',
    'multimedia': [{'rank': 0,
      'subtype': 'xlarge',
      'caption': None,
      'credit': None,
      'type': 'image',
      'url': 'images/2024/01/25/multimedia/26ambriefing-asia-nl_Kim-Jong-un/26ambriefing-asia-nl_Kim-Jong-un-articleLarge.jpg',
      'height': 600,
      'width': 600,
      'legacy': {'xlarge': 'images/2024/01/25/multimedia/26ambriefi